# Gesture Detection Model with interpolated data

## 1 Feature engineering

### 1.1 Import libraries and transformers  

In [1]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from keras.utils import to_categorical
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [2]:
import os
cwd = os.getcwd()

counter = 0
print(counter, cwd)
while not cwd.endswith("drone_steering") and counter < 10:
    os.chdir('..')
    cwd = os.getcwd()
    counter = counter + 1
    print(counter, cwd)

0 C:\Users\p.schambach\Desktop\DSR\drone_steering\models\playground
1 C:\Users\p.schambach\Desktop\DSR\drone_steering\models
2 C:\Users\p.schambach\Desktop\DSR\drone_steering


In [3]:
from app_local.module import DataEnsembler, LabelGenerator, GestureTransformer, DataResampler

### 1.2  Load data

In [4]:
de = DataEnsembler(ms_per_frame=120)
de.investigate_available_datafiles(data_dir='data/gesture/', is_frame_based = True)
combined_data_files_df=de.combined_data_files_df
combined_data_files_df

,filename_features,filename_labels
0,features_flip_c_01_120.csv,labels_flip_c_01.csv
1,features_flip_c_02_120.csv,labels_flip_c_02.csv
2,features_flip_c_03_120.csv,labels_flip_c_03.csv
3,features_flip_l_01_120.csv,labels_flip_l_01.csv
4,features_flip_l_02_120.csv,labels_flip_l_02.csv
5,features_flip_l_03_120.csv,labels_flip_l_03.csv
6,features_flip_p_01_120.csv,labels_flip_p_01.csv
7,features_land_c_01_120.csv,labels_land_c_01.csv
8,features_land_c_02_120.csv,labels_land_c_02.csv
9,features_land_c_03_120.csv,labels_land_c_03.csv


In [5]:
de.load_data()

In [6]:
de.video_stats_pattern

'(?P<movement>[a-z]*)_(?P<person>[a-z]*)_(?P<filenum>\\d*)'

### 1. 5 Interpolated data assembling

In [8]:
# the training data can then be assembled as usual
de.assemble_data(tolerance_range=3)
de.display_information()

i: 0 	shape X: (603, 18, 16) 	shape y: (603,) 	count: 127
i: 1 	shape X: (519, 18, 16) 	shape y: (519,) 	count: 115
i: 2 	shape X: (498, 18, 16) 	shape y: (498,) 	count: 149
i: 3 	shape X: (650, 18, 16) 	shape y: (650,) 	count: 276
i: 4 	shape X: (508, 18, 16) 	shape y: (508,) 	count: 173
i: 5 	shape X: (466, 18, 16) 	shape y: (466,) 	count: 174
i: 6 	shape X: (601, 18, 16) 	shape y: (601,) 	count: 185
i: 7 	shape X: (440, 18, 16) 	shape y: (440,) 	count: 149
i: 8 	shape X: (490, 18, 16) 	shape y: (490,) 	count: 130
i: 9 	shape X: (476, 18, 16) 	shape y: (476,) 	count: 140
i: 10 	shape X: (492, 18, 16) 	shape y: (492,) 	count: 139
i: 11 	shape X: (436, 18, 16) 	shape y: (436,) 	count: 101
i: 12 	shape X: (466, 18, 16) 	shape y: (466,) 	count: 146
i: 13 	shape X: (551, 18, 16) 	shape y: (551,) 	count: 155
i: 14 	shape X: (550, 18, 16) 	shape y: (550,) 	count: 142
i: 15 	shape X: (549, 18, 16) 	shape y: (549,) 	count: 145
i: 16 	shape X: (541, 18, 16) 	shape y: (541,) 	count: 112
i: 17 	

### 1. 7 Imbalance data treatment

In [19]:
dr = DataResampler()
dr.fit(de.X, de.y)
dr.restrict_zero_labels(0)
X = dr.X
y = dr.y
dr.display_information()

-------------------------------------
Current number of zero-labels: 13032
New number of zero-labels: 0


,abs_orig,pct_orig,abs_current,pct_current
0,13032,71.65,0,0
1,825,4.54,825,16
2,680,3.74,680,13.19
3,1199,6.59,1199,23.25
4,833,4.58,833,16.16
5,814,4.48,814,15.79
6,805,4.43,805,15.61


### 1.3  Data normalization

In [20]:
gt = GestureTransformer(feature_names = list(de.feature_names),byrow=True)
X = gt.transform(X)
y_enc = to_categorical(y)

## 2 Train model

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_enc, test_size=0.2, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(4124, 18, 16) (1032, 18, 16) (4124, 7) (1032, 7)


### 2.2 Define and compile model

In [23]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM

In [24]:
n_timesteps = X_train.shape[1] # here 17
n_features = X_train.shape[2] # here 16
n_outputs =  y_enc.shape[1] # here 7 (number of labels)

model = Sequential()
model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
model.add(Dropout(0.4))
model.add(Dense(100, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 100)               46800     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_8 (Dense)              (None, 64)                6464      
_________________________________________________________________
dense_9 (Dense)              (None, 7)                 455       
Total params: 63,819
Trainable params: 63,819
Non-trainable params: 0
_________________________________________________________________


### 2.3 Fit and validate model

In [25]:
history_interpol = model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1, validation_split=0.2)

Train on 3299 samples, validate on 825 samples
Epoch 1/10
3299/3299 [==============================] - 2s 525us/step - loss: 0.6126 - acc: 0.8039 - val_loss: 0.0475 - val_acc: 0.9879
Epoch 2/10
3299/3299 [==============================] - 1s 254us/step - loss: 0.0540 - acc: 0.9876 - val_loss: 0.0478 - val_acc: 0.9842
Epoch 3/10
3299/3299 [==============================] - 1s 258us/step - loss: 0.0340 - acc: 0.9942 - val_loss: 0.0076 - val_acc: 0.9976
Epoch 4/10
3299/3299 [==============================] - 1s 258us/step - loss: 0.0135 - acc: 0.9976 - val_loss: 0.0088 - val_acc: 0.9964
Epoch 5/10
3299/3299 [==============================] - 1s 258us/step - loss: 0.0138 - acc: 0.9976 - val_loss: 0.0114 - val_acc: 0.9976
Epoch 6/10
3299/3299 [==============================] - 1s 257us/step - loss: 0.0128 - acc: 0.9979 - val_loss: 0.0116 - val_acc: 0.9952
Epoch 7/10
3299/3299 [==============================] - 1s 258us/step - loss: 0.0130 - acc: 0.9973 - val_loss: 0.0126 - val_acc: 0.9976
E

In [ ]:
r = range(1, len(history_interpol.history["acc"]) + 1)

plt.figure(figsize=(20, 5))

plt.subplot(1, 2, 1)
plt.plot(r, history_interpol.history["acc"], label="acc")
plt.plot(r, history_interpol.history["val_acc"], label="val_acc")
plt.legend()
plt.title("Accuracy")

plt.subplot(1, 2, 2)
plt.plot(r, history_interpol.history["loss"], label="loss")
plt.plot(r, history_interpol.history["val_loss"], label="val_loss")
plt.legend()
plt.title("Loss")

plt.show()
plt.close()

In [ ]:
y_test = to_categorical(y_test)
eval_results = np.round(model.evaluate(X_test, y_test, batch_size=32, verbose=1),2)
print("Loss:     " + str(eval_results[0]))
print("Accuracy: " + str(eval_results[1]))

In [ ]:
model.

## ROC analysis

In [ ]:
counter_F_neg=0
counter_F_pos=0
for i in range(X_test.shape[0]):
    if (model.predict_classes(X_test[i:(i+1),:,:])[0]!=np.nonzero(y_test[i])[0][0]):
        if model.predict_classes(X_test[i:(i+1),:,:])[0]==0:
            counter_F_neg=counter_F_neg+1
            print("False_negative i:",i,\
              "\tPred:",model.predict_classes(X_test[i:(i+1),:,:])[0],\
              "\tActual:",np.nonzero(y_test[i])[0][0],\
              "\tProbability:",round(np.max(model.predict_proba(X_test[i:(i+1),:,:])),2),)          
        else:
            counter_F_pos=counter_F_pos+1
            print("False_positive i:",i,\
              "\tPred:",model.predict_classes(X_test[i:(i+1),:,:])[0],\
              "\tActual:",np.nonzero(y_test[i])[0][0],\
              "\tProbability:",round(np.max(model.predict_proba(X_test[i:(i+1),:,:])),2))
print("\ncounter_F_neg:",counter_F_neg) 
print("\ncounter_F_pos:",counter_F_pos) 

In [ ]:
for i in range(X_test.shape[0]):
    if (model.predict_classes(X_test[i:(i+1),:,:])[0]==np.nonzero(y_test[i])[0][0]):
        print("Index:",i,\
              "\tPred:",model.predict_classes(X_test[i:(i+1),:,:])[0],\
              "\tActual:",np.nonzero(y_test[i])[0][0],\
              "\tProbability:",round(np.max(model.predict_proba(X_test[i:(i+1),:,:])),2))

In [93]:
yvar = y_test.copy()
n = yvar.shape[0]

label = np.zeros(n)
pred = np.zeros(n)
prob = np.zeros((n,7))

for i in range(n):
    label[i] = np.argmax(yvar[i,:])
    pred[i] = model.predict_classes(X_test[i:(i+1),:,:])
    prob[i,:] = model.predict_proba(X_test[i:(i+1),:,:])
    if i % 100 == 0:
        print(i)
    
print(label, pred, prob) 

0
100
200
300
400
500
600
700
800
900
1000
[5. 3. 6. ... 6. 2. 3.] [5. 3. 6. ... 6. 2. 3.] [[3.56156988e-08 5.94293124e-06 8.05360571e-07 ... 2.61384741e-07
  9.99978065e-01 6.64416848e-06]
 [4.74521422e-10 1.28134543e-08 1.29390639e-06 ... 7.97221289e-10
  1.03037485e-06 7.16488273e-08]
 [3.67486682e-05 2.06999592e-02 1.45025624e-04 ... 1.25361737e-04
  2.14560176e-04 9.78569388e-01]
 ...
 [9.04814840e-07 1.03262428e-03 9.48177330e-05 ... 2.27165197e-06
  1.87810638e-05 9.98832881e-01]
 [2.09816990e-11 5.35125764e-07 9.99982238e-01 ... 4.79079840e-12
  6.20076008e-08 1.48865583e-05]
 [6.26201868e-11 2.01464645e-09 3.25696931e-07 ... 1.06451063e-10
  3.45106287e-07 1.01039435e-08]]


In [98]:
r = 0.999
for i in range(1,7):
    print(i, len(label[label==i]), len(pred[pred==i]), (prob[:,i]>r).sum() )

1 147 145 132
2 130 130 130
3 247 247 245
4 170 170 151
5 169 169 169
6 169 171 7


In [141]:
n = de.X.shape[0]
v = np.zeros(n)
s = np.zeros(n)

for i in range(n):
    v[i] = np.abs(de.X[i,:,:].var(axis = 0)).sum()
    s[i] = np.abs(np.diff(de.X[i,:,:], axis = 0)).sum()

In [111]:
for i in range(7):
    print(i, v[de.y == i].sum()/len(de.y[de.y==i]))

0 5986.171276338981
1 16359.31188714042
2 5618.657622171907
3 13710.80719554805
4 5106.582079900341
5 5264.4543143634855
6 17351.097226036945


In [129]:
tvar = 3000.0
n = (v >= tvar).sum()
for i in range(7):
    ni = len(v[(v >= tvar) & (de.y == i)])
    print(i, ni, len(de.y[de.y==i]), np.round(100 * ni/ n,2))

0 7221 13032 58.63
1 820 825 6.66
2 669 680 5.43
3 1199 1199 9.73
4 829 833 6.73
5 777 814 6.31
6 802 805 6.51


In [142]:
for i in range(7):
    print(i, s[de.y == i].sum()/len(de.y[de.y==i]))

0 970.1123599622166
1 1569.3360335205427
2 1415.9102927221625
3 1834.4517439551632
4 1019.2480728470169
5 996.2701382758934
6 1706.125898798009


In [147]:
svar = 800.0
n = (s >= svar).sum()
for i in range(7):
    ni = len(s[(s >= svar) & (de.y == i)])
    print(i, ni, len(de.y[de.y==i]), np.round(100 * ni/ n,2))

0 7056 13032 58.31
1 824 825 6.81
2 680 680 5.62
3 1199 1199 9.91
4 803 833 6.64
5 733 814 6.06
6 805 805 6.65


### 2.4 Save model

In [ ]:
from keras.models import load_model
import h5py

In [ ]:
# save model, delete current reference and re-load it from file
model.save('models/model_gesture_interpolation_' + str(new_frmlen) + '.h5')
del model
model = load_model('005_model_gesture-interpolation.h5')

In [ ]:
type(model)